In [1]:
import pandas as pd
import numpy as np
from rfm import RFM

Загружаем файлы с транзакциями и клиентами

In [2]:
df_clients = pd.read_csv("./clients_small.csv", sep=';')
df_transactions = pd.read_csv("./transactions_small.csv", sep=';')


In [3]:
df_clients.head(3)

,customer_id,card_id,region_name,sex,birth_date
0,02F4DB34C5696A668068CA3C1DE7C2D5,6600B4FDA036A6F00D464541EAD3B21F,Москва,F,1938-08-18
1,0308713D02D676920F2961E2A84B4516,1FB9344B8661069910D184EF19C448C7,Сахалинская область,F,1990-03-26
2,032672FDC2CF85213079F83E447AF75B,4C8AD60064AF412753EB54EAFCD8C59D,Самарская область,F,1989-01-13


In [4]:
df_transactions.head(3)

,customer_id,card_id,tr_datetime,tr_normalized_amount,currency,mcc,merchant_name,merchant_city
0,634E7564849D27B3C452229E405629A6,52CD1B87531555F5611C663562FC686D,2019-09-07,10.00,810,5999,i-bank ┴шыaщэ >moscow ru,moscow
1,B20A638E7EB926C63102144596F38C71,9D4F087597B65028953FBE8A8FBF48DE,2019-09-20,313.04,810,5411,dixy moskva ru,moscow
2,125EBE0A38873D4888408F85877282EF,B8859E8DB421C73B785521A6252A36CB,2019-09-23,250.00,810,5999,i-bank teыe 2 >moscow ru,moscow


Пробуем найти косячные данные типа "i-bank ┴шыaщэ" в столбце merchant_name \
Для этого создаем функцию, которая находит не Unicode символы 

In [5]:
r = RFM(df_transactions, customer_id='customer_id', transaction_date='tr_datetime', amount='tr_normalized_amount')

In [6]:
df_transactions['tr_datetime'] = pd.to_datetime(df_transactions['tr_datetime'])

In [7]:
df_clients_rfm = r.rfm_table

In [8]:
df_clients_rfm.head(5)

,customer_id,recency,frequency,monetary_value,r,f,m,rfm_score,segment
0,13A873ECF427403915BB58EEF5A0D35B,0,512,1402853.97,5,5,5,555,Champions
1,42E548AE6B3D1A06DA03AADE06F3320D,0,274,226011.60,5,5,5,555,Champions
2,05347FF04C750066BCDA337B4FB3BA71,0,180,256207.90,5,5,5,555,Champions
3,0BEE43A4B9A92C7899073DEA8FD8C7E2,0,273,157289.96,5,5,5,555,Champions
4,545A66B2E91CC776FCCDF285DE690FB3,0,201,196780.88,5,5,5,555,Champions


In [9]:
df_clients_rfm.pivot_table(columns=['r'],index=['f'], values = ['customer_id'], aggfunc= 'count',fill_value=0).reset_index()

f customer_id                    
r              1    2    3    4    5
0  1         542  220  126  100   86
1  2         242  280  200  179  173
2  3         152  255  231  199  237
3  4          89  204  247  261  273
4  5          49  115  270  335  305

In [10]:
df_clients = df_clients[['customer_id','region_name','sex','birth_date']].merge(df_clients_rfm[['r','f','m','rfm_score']], left_index=True, right_index=True)


In [11]:
#Меняем тип даты рождения в датафрейме  
df_clients['birth_date'] = pd.to_datetime(df_clients['birth_date'], errors='coerce')

In [12]:
df_clients.dtypes

customer_id            object
region_name            object
sex                    object
birth_date     datetime64[ns]
r                       int64
f                       int64
m                       int64
rfm_score              object
dtype: object

In [13]:
df_clients['birth_date_bin'] = pd.cut(df_clients['birth_date'], bins=5, labels=False) + 1

In [14]:
#Заполяем бины для тех кто у кого нет даты рождения нулями 
df_clients.loc[df_clients['birth_date_bin'].isnull(),'birth_date_bin'] = 0

In [15]:
df_clients['birth_date_bin'].unique()

array([1., 5., 4., 0., 2., 3.])

In [16]:
df_clients

,customer_id,region_name,sex,birth_date,r,f,m,rfm_score,birth_date_bin
0,02F4DB34C5696A668068CA3C1DE7C2D5,Москва,F,1938-08-18,5,5,5,555,1.0
1,0308713D02D676920F2961E2A84B4516,Сахалинская область,F,1990-03-26,5,5,5,555,5.0
2,032672FDC2CF85213079F83E447AF75B,Самарская область,F,1989-01-13,5,5,5,555,4.0
3,048C34209D21671371750B9E045F618F,Самарская область,M,1984-05-04,5,5,5,555,4.0
4,056435AF351D99DEE13F5119F785E6CB,Республика Башкортостан,F,1990-11-24,5,5,5,555,5.0
...,...,...,...,...,...,...,...,...,...
5365,E5C4718102EE4121CC4BF9528B7C5CA5,Приморский край,M,1974-05-19,1,1,1,111,3.0
5366,E885BB964AC47E62B8BC7E415A0F6F56,Кемеровская область,F,1979-04-08,1,1,1,111,4.0
5367,EA7F57D715AB1E1A5A538DC1CD5948C2,Москва,M,1986-04-10,1,1,1,111,4.0
5368,EB79175A843797FBF57310879AD8068D,Красноярский край,F,1984-02-10,1,1,1,111,4.0
